<a href="https://colab.research.google.com/github/nah90/CS777_PySpark-MLlib/blob/main/MET_CS_777_Assignment_5_Horak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --ignore-installed -q pyspark==3.2.1

In [ ]:
import re
import sys
import numpy as np
from operator import add
from timeit import default_timer as timer
import time


from operator import add
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
vocab_size = 5000

In [ ]:
# ! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTrainingData.txt.bz2

# Use this code to read the data
courtFile1 = 'SmallTrainingData.txt.bz2'
corpus1 = sc.textFile(courtFile1, 1)

In [ ]:
#Task 1
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml import Pipeline

task_1_timer = timer()

keyAndTextTrain = corpus1.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTraindf = spark.createDataFrame(keyAndTextTrain).toDF("id","label","text").cache()

tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='words')
cv = CountVectorizer(inputCol=remover.getOutputCol(), outputCol='raw_features', vocabSize=vocab_size)
idf = IDF(inputCol=cv.getOutputCol(), outputCol='features')

pipeline1 = Pipeline(stages=[tokenizer, remover, cv, idf])
model1 = pipeline1.fit(keyLabelTextTraindf)

keyLabelTextTraindf.unpersist()

print("Total time needed to vectorize data in seconds: ", timer() - task_1_timer)
print("First 10 words from our vocabulary: 5k positions: ", model1.stages[-2].vocabulary[:10])

Total time needed to vectorize data in seconds:  24.39843286699761
First 10 words from our vocabulary: 5k positions:  ['also', 'first', 'one', 'new', 'two', 'may', 'time', 'made', 'many', 'used']


In [ ]:
#Task 2
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

# ! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTrainingData.txt.bz2

# Use this code to read the data
courtFile1 = 'SmallTrainingData.txt.bz2'
corpus1 = sc.textFile(courtFile1, 1)

keyAndTextTrain = corpus1.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTraindf = spark.createDataFrame(keyAndTextTrain).toDF("id","label","text").cache()

task_2_timer = timer()
training_2_timer = timer()

tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='words')
cv = CountVectorizer(inputCol=remover.getOutputCol(), outputCol='raw_features', vocabSize=vocab_size)
idf = IDF(inputCol=cv.getOutputCol(), outputCol='features')
lr = LogisticRegression(maxIter=20, labelCol='label', featuresCol=idf.getOutputCol())

pipeline2 = Pipeline(stages=[tokenizer, remover, cv, idf, lr])
model2 = pipeline2.fit(keyLabelTextTraindf)

keyLabelTextTraindf.unpersist()

print("Total time needed to train model in seconds: ", timer() - training_2_timer)

Total time needed to train model in seconds:  30.204910868997104


In [ ]:
# ! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTestingData.txt.bz2

# Use this code to read the data
courtFile2 = 'SmallTestingData.txt.bz2'
corpus2 = sc.textFile(courtFile2, 1)

evaluation_2_timer = timer()

keyAndTextTest = corpus2.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTestdf = spark.createDataFrame(keyAndTextTest).toDF("id","label","text").cache()

In [ ]:
predictions2 = model2.transform(keyLabelTextTestdf)

keyLabelTextTestdf.unpersist()

predictionAndLabels2 = predictions2.select("label","prediction").rdd.map(lambda x: (float(x[0]), float(x[1]))).cache()

print("Total time needed to evaluate test data in seconds: ", timer() - evaluation_2_timer)

Total time needed to evaluate test data in seconds:  0.4945748509999248


In [ ]:
metrics_2_timer = timer()

metrics2 = MulticlassMetrics(predictionAndLabels2)

predictionAndLabels2.unpersist()

print("F1 Score: ", metrics2.fMeasure(1.0))
print("Confusion Matrix: ")
print(metrics2.confusionMatrix().toArray().astype(int))
print('')

print("Total time needed to calculate performance metrics in seconds: ", timer() - metrics_2_timer)
print("Total time needed to execute Task 2 in seconds: ", timer() - task_2_timer)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


F1 Score:  1.0
Confusion Matrix: 
[[20  0]
 [ 0 20]]

Total time needed to calculate performance metrics in seconds:  0.5912671879996196
Total time needed to execute Task 2 in seconds:  31.346847290998994


In [ ]:
#Task 3
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import LinearSVC
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

# ! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTrainingData.txt.bz2

# Use this code to read the data
courtFile1 = 'SmallTrainingData.txt.bz2'
corpus1 = sc.textFile(courtFile1, 1)

keyAndTextTrain = corpus1.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTraindf = spark.createDataFrame(keyAndTextTrain).toDF("id","label","text").cache()

task_3_timer = timer()
training_3_timer = timer()

tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='words')
cv = CountVectorizer(inputCol=remover.getOutputCol(), outputCol='raw_features', vocabSize=vocab_size)
idf = IDF(inputCol=cv.getOutputCol(), outputCol='features')
svm = LinearSVC(maxIter=20, labelCol='label', featuresCol=idf.getOutputCol())

pipeline3 = Pipeline(stages=[tokenizer, remover, cv, idf, svm])
model3 = pipeline3.fit(keyLabelTextTraindf)

keyLabelTextTraindf.unpersist()

print("Total time needed to train model in seconds: ", timer() - training_3_timer)

Total time needed to train model in seconds:  26.99593031599943


In [ ]:
# ! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTestingData.txt.bz2

# Use this code to read the data
courtFile2 = 'SmallTestingData.txt.bz2'
corpus2 = sc.textFile(courtFile2, 1)

evaluation_3_timer = timer()

keyAndTextTest = corpus2.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTestdf = spark.createDataFrame(keyAndTextTest).toDF("id","label","text").cache()

In [ ]:
predictions3 = model3.transform(keyLabelTextTestdf)

keyLabelTextTestdf.unpersist()

predictionAndLabels3 = predictions3.select("label","prediction").rdd.map(lambda x: (float(x[0]), float(x[1]))).cache()

print("Total time needed to evaluate test data in seconds: ", timer() - evaluation_3_timer)

Total time needed to evaluate test data in seconds:  0.34237991900226916


In [ ]:
metrics_3_timer = timer()

metrics3 = MulticlassMetrics(predictionAndLabels3)

predictionAndLabels3.unpersist()

print("F1 Score: ", metrics3.fMeasure(1.0))
print("Confusion Matrix: ")
print(metrics3.confusionMatrix().toArray().astype(int))
print('')

print("Total time needed to calculate performance metrics in seconds: ", timer() - metrics_3_timer)
print("Total time needed to execute Task 3 in seconds: ", timer() - task_3_timer)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


F1 Score:  1.0
Confusion Matrix: 
[[20  0]
 [ 0 20]]

Total time needed to calculate performance metrics in seconds:  0.500955994000833
Total time needed to execute Task 3 in seconds:  27.91153830300027


In [ ]:
#Task 4
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, ChiSqSelector
from pyspark.ml.classification import LogisticRegression, LinearSVC
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

# ! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTrainingData.txt.bz2

# Use this code to read the data
courtFile1 = 'SmallTrainingData.txt.bz2'
corpus1 = sc.textFile(courtFile1, 1)

keyAndTextTrain = corpus1.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTraindf = spark.createDataFrame(keyAndTextTrain).toDF("id","label","text").cache()

reduced_features = 200

In [ ]:
#Subtask A - Logistic Regression

task_4a_timer = timer()
training_4a_timer = timer()

tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='words')
cv = CountVectorizer(inputCol=remover.getOutputCol(), outputCol='raw_features', vocabSize=vocab_size)
idf = IDF(inputCol=cv.getOutputCol(), outputCol='features')
selector = ChiSqSelector(numTopFeatures=reduced_features, labelCol='label', featuresCol=idf.getOutputCol(), outputCol='selected_features')
lr = LogisticRegression(maxIter=20, labelCol='label', featuresCol=selector.getOutputCol())

pipeline4a = Pipeline(stages=[tokenizer, remover, cv, idf, selector, lr])
model4a = pipeline4a.fit(keyLabelTextTraindf)

keyLabelTextTraindf.unpersist()

print("Total time needed to train model in seconds: ", timer() - training_4a_timer)

Total time needed to train model in seconds:  38.35479265699905


In [ ]:
# ! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTestingData.txt.bz2

# Use this code to read the data
courtFile2 = 'SmallTestingData.txt.bz2'
corpus2 = sc.textFile(courtFile2, 1)

evaluation_4a_timer = timer()

keyAndTextTest = corpus2.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTestdf = spark.createDataFrame(keyAndTextTest).toDF("id","label","text").cache()

In [ ]:
predictions4a = model4a.transform(keyLabelTextTestdf)

keyLabelTextTestdf.unpersist()

predictionAndLabels4a = predictions4a.select("label","prediction").rdd.map(lambda x: (float(x[0]), float(x[1]))).cache()

print("Total time needed to evaluate test data in seconds: ", timer() - evaluation_4a_timer)

Total time needed to evaluate test data in seconds:  0.3559647359979863


In [ ]:
metrics_4a_timer = timer()

metrics4a = MulticlassMetrics(predictionAndLabels4a)

predictionAndLabels4a.unpersist()

print("F1 Score: ", metrics4a.fMeasure(1.0))
print("Confusion Matrix: ")
print(metrics4a.confusionMatrix().toArray().astype(int))
print('')

print("Total time needed to calculate performance metrics in seconds: ", timer() - metrics_4a_timer)
print("Total time needed to execute Task 4a in seconds: ", timer() - task_4a_timer)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


F1 Score:  0.888888888888889
Confusion Matrix: 
[[20  4]
 [ 0 16]]

Total time needed to calculate performance metrics in seconds:  0.5742057980023674
Total time needed to execute Task 4a in seconds:  39.340379474000656


In [ ]:
courtFile1 = 'SmallTrainingData.txt.bz2'
corpus3 = sc.textFile(courtFile1, 1)

keyAndTextTrain = corpus3.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTraindf = spark.createDataFrame(keyAndTextTrain).toDF("id","label","text").cache()

In [ ]:
#Subtask B - Support Vector Machine

task_4b_timer = timer()
training_4b_timer = timer()

tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='words')
cv = CountVectorizer(inputCol=remover.getOutputCol(), outputCol='raw_features', vocabSize=vocab_size)
idf = IDF(inputCol=cv.getOutputCol(), outputCol='features')
selector = ChiSqSelector(numTopFeatures=reduced_features, labelCol='label', featuresCol=idf.getOutputCol(), outputCol='selected_features')
svm = LinearSVC(maxIter=20, labelCol='label', featuresCol=selector.getOutputCol())

pipeline4b = Pipeline(stages=[tokenizer, remover, cv, idf, selector, svm])
model4b = pipeline4b.fit(keyLabelTextTraindf)

keyLabelTextTraindf.unpersist()

print("Total time needed to train model in seconds: ", timer() - training_4b_timer)

Total time needed to train model in seconds:  37.95938682600172


In [ ]:
evaluation_4b_timer = timer()

# ! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTestingData.txt.bz2

# Use this code to read the data
courtFile2 = 'SmallTestingData.txt.bz2'
corpus4 = sc.textFile(courtFile2, 1)

evaluation_4a_timer = timer()

keyAndTextTest = corpus4.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))\
  .map(lambda x: (x[0], int(x[0].startswith("AU")),x[1]))

keyLabelTextTestdf = spark.createDataFrame(keyAndTextTest).toDF("id","label","text").cache()

In [ ]:
predictions4b = model4b.transform(keyLabelTextTestdf)

keyLabelTextTestdf.unpersist()

predictionAndLabels4b = predictions4b.select("label","prediction").rdd.map(lambda x: (float(x[0]), float(x[1]))).cache()

print("Total time needed to evaluate test data in seconds: ", timer() - evaluation_4b_timer)

Total time needed to evaluate test data in seconds:  0.3507673029998841


In [ ]:
metrics_4b_timer = timer()

metrics4b = MulticlassMetrics(predictionAndLabels4b)

predictionAndLabels4b.unpersist()

print("F1 Score: ", metrics4b.fMeasure(1.0))
print("Confusion Matrix: ")
print(metrics4b.confusionMatrix().toArray().astype(int))
print('')

print("Total time needed to calculate performance metrics in seconds: ", timer() - metrics_4b_timer)
print("Total time needed to execute Task 4a in seconds: ", timer() - task_4b_timer)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


F1 Score:  0.888888888888889
Confusion Matrix: 
[[20  4]
 [ 0 16]]

Total time needed to calculate performance metrics in seconds:  0.6152691170027538
Total time needed to execute Task 4a in seconds:  38.96047091299988
